<a href="https://colab.research.google.com/github/eshal26/store-sales-forecast/blob/main/Store_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Store Sales Forecast </h1>

<h2>Data Loading and installing packages</h2>

In [ ]:
import pandas as pd
from prophet import Prophet
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go
from prophet.diagnostics import cross_validation, performance_metrics

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [ ]:
df.shape

(3000888, 6)

In [ ]:
df.dtypes

,0
id,int64
date,object
store_nbr,int64
family,object
sales,float64
onpromotion,int64


In [ ]:
df=df[['date','sales']]

In [ ]:
df.head()

,date,sales
0,2013-01-01,0.0
1,2013-01-01,0.0
2,2013-01-01,0.0
3,2013-01-01,0.0
4,2013-01-01,0.0


<h2>Preparing data for prophet</h2>

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df = df.groupby('date',as_index=False).mean()
print(df.head())

        date       sales
0 2013-01-01    1.409438
1 2013-01-02  278.390807
2 2013-01-03  202.840197
3 2013-01-04  198.911154
4 2013-01-05  267.873244


In [ ]:
print(df.shape)

(1684, 2)


In [ ]:
fig = px.line(df, x='date', y='sales', title='Sales Over Time')
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [ ]:
df.columns = ['ds','y']

In [ ]:
df.head()

,ds,y
0,2013-01-01,1.409438
1,2013-01-02,278.390807
2,2013-01-03,202.840197
3,2013-01-04,198.911154
4,2013-01-05,267.873244


<h2>Prediction using prophet</h2>

In [ ]:
model=Prophet()

In [ ]:
model.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/gpjj3ka7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/83ssl4o4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=39666', 'data', 'file=/tmp/tmp5h7_1j51/gpjj3ka7.json', 'init=/tmp/tmp5h7_1j51/83ssl4o4.json', 'output', 'file=/tmp/tmp5h7_1j51/prophet_model_ap77lr2/prophet_model-20240824105459.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:54:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:54:59 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
model.component_modes

{'additive': ['yearly',
  'weekly',
  'additive_terms',
  'extra_regressors_additive',
  'holidays'],
 'multiplicative': ['multiplicative_terms', 'extra_regressors_multiplicative']}

In [ ]:
future_dates = model.make_future_dataframe(periods=365,freq='d',include_history=True)
future_dates.shape

(2049, 1)

In [ ]:
future_dates.head()

,ds
0,2013-01-01
1,2013-01-02
2,2013-01-03
3,2013-01-04
4,2013-01-05


In [ ]:
prediction = model.predict(future_dates)

In [ ]:
prediction.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2013-01-01,187.844545,136.550239,292.490874,187.844545,187.844545,24.379850,24.379850,24.379850,-38.256745,-38.256745,-38.256745,62.636595,62.636595,62.636595,0.0,0.0,0.0,212.224395
1,2013-01-02,188.053938,148.953112,293.891072,188.053938,188.053938,31.120387,31.120387,31.120387,-24.813380,-24.813380,-24.813380,55.933767,55.933767,55.933767,0.0,0.0,0.0,219.174325
2,2013-01-03,188.263331,86.008069,245.991559,188.263331,188.263331,-24.783170,-24.783170,-24.783170,-74.003953,-74.003953,-74.003953,49.220783,49.220783,49.220783,0.0,0.0,0.0,163.480162
3,2013-01-04,188.472725,119.959564,279.597298,188.472725,188.472725,10.575655,10.575655,10.575655,-32.022077,-32.022077,-32.022077,42.597732,42.597732,42.597732,0.0,0.0,0.0,199.048379
4,2013-01-05,188.682118,218.239913,375.959164,188.682118,188.682118,111.378516,111.378516,111.378516,75.219504,75.219504,75.219504,36.159012,36.159012,36.159012,0.0,0.0,0.0,300.060633


In [ ]:
trace_open = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat"],
    mode = 'lines',
    name="Forecast"
)
trace_high = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_upper"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Higher uncertainty interval"
)
trace_low = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_lower"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Lower uncertainty interval"
)
trace_close = go.Scatter(
    x = df["ds"],
    y = df["y"],
    name="Data values"
)

#make list for all three scattle objects.
data = [trace_open,trace_high,trace_low,trace_close]
# Construct a new Layout object.
#title - It will display string as a title of graph
layout = go.Layout(title="Power consumption forecasting")
#A list or tuple of trace instances (e.g. [Scatter(…), Bar(…)]) or A single trace instance (e.g. Scatter(…), Bar(…), etc.)
#A list or tuple of dicts of string/value properties where: - The ‘type’ property specifies the trace type.

fig = go.Figure(data=data)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:

fig = go.Figure([go.Scatter(x=df['ds'], y=df['y'],mode='lines',
                    name='Actual')])
#You can add traces using an Express plot by using add_trace
fig.add_trace(go.Scatter(x=prediction['ds'], y=prediction['yhat'],
                   mode='lines+markers',
                    name='predicted'))
#To display a figure using the renderers framework, you call the .show() method on a graph object figure, or pass the figure to the plotly.io.show function.
#With either approach, plotly.py will display the figure using the current default renderer(s).
fig.show()


<h2>Parameter Grid</h2>
fining the best parameters for our model

In [ ]:

# Define the parameter grid
param_grid = {
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'fourier_order': [60, 70, 80],
}

# Prepare to store the results
best_params = None
best_mae = float('inf')

# Loop over all combinations of the parameters
for cps in param_grid['changepoint_prior_scale']:
    for fo in param_grid['fourier_order']:

        # Initialize and fit the model with the current parameters
        model = Prophet(changepoint_prior_scale=cps, daily_seasonality=True)
        model.add_seasonality(name='yearly', period=365, fourier_order=fo)
        model.fit(df)

        # Cross-validate the model
        df_cv = cross_validation(model, initial='365 days', period='180 days', horizon='365 days')

        # Calculate performance metrics
        df_p = performance_metrics(df_cv, rolling_window=1)

        # Check if the current model is the best one
        if df_p['mae'].mean() < best_mae:
            best_mae = df_p['mae'].mean()
            best_params = {'changepoint_prior_scale': cps, 'fourier_order': fo}

# Output the best parameters
print(f"Best Parameters: {best_params}")
print(f"Best MAE: {best_mae:.3f}")


INFO:prophet:Found custom seasonality named 'yearly', disabling built-in 'yearly' seasonality.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/gzry9maq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/kssva_nx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=62087', 'data', 'file=/tmp/tmp5h7_1j51/gzry9maq.json', 'init=/tmp/tmp5h7_1j51/kssva_nx.json', 'output', 'file=/tmp/tmp5h7_1j51/prophet_modeli3xmlhd7/prophet_model-20240824110531.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:05:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:05:31 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 6 forecasts with cutoffs between 2014-02-27 00:00:00 and 2016-08-15 00:00:00


  0%|          | 0/6 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/ouydtglz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/433te4sd.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=36125', 'data', 'file=/tmp/tmp5h7_1j51/ouydtglz.json', 'init=/tmp/tmp5h7_1j51/433te4sd.json', 'output', 'file=/tmp/tmp5h7_1j51/prophet_model1mell5vb/prophet_model-20240824110532.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:05:32 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:05:32 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/vtesb5_t.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/ok5cptvg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

  0%|          | 0/6 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/booch0on.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/7qd_3no1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=29706', 'data', 'file=/tmp/tmp5h7_1j51/booch0on.json', 'init=/tmp/tmp5h7_1j51/7qd_3no1.json', 'output', 'file=/tmp/tmp5h7_1j51/prophet_modelbi0l3ss8/prophet_model-20240824110538.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:05:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:05:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/nugvlb1m.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/vgoikplj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

...

  0%|          | 0/6 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/tsycf1dp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/9_8rb7hv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=61656', 'data', 'file=/tmp/tmp5h7_1j51/tsycf1dp.json', 'init=/tmp/tmp5h7_1j51/9_8rb7hv.json', 'output', 'file=/tmp/tmp5h7_1j51/prophet_modelezm5sco4/prophet_model-20240824110644.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:06:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:06:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/3in8b1rg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/ejgb_xtg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

  0%|          | 0/6 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/mo798zl1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/_wdrb7jj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=42111', 'data', 'file=/tmp/tmp5h7_1j51/mo798zl1.json', 'init=/tmp/tmp5h7_1j51/_wdrb7jj.json', 'output', 'file=/tmp/tmp5h7_1j51/prophet_modelmmjsl_2o/prophet_model-20240824110704.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:07:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:07:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/p51xoc8n.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/ry__ggci.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Best Parameters: {'changepoint_prior_scale': 0.01, 'fourier_order': 70}
Best MAE: 61.012


<h2>Predictions using new parameters</h2>

In [ ]:
model1 = Prophet(changepoint_prior_scale=0.01, daily_seasonality=70)
model1.add_seasonality(name='yearly', period=365, fourier_order=fo)
model1.fit(df)

INFO:prophet:Found custom seasonality named 'yearly', disabling built-in 'yearly' seasonality.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/agg7y_3t.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5h7_1j51/b66w09hf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10239', 'data', 'file=/tmp/tmp5h7_1j51/agg7y_3t.json', 'init=/tmp/tmp5h7_1j51/b66w09hf.json', 'output', 'file=/tmp/tmp5h7_1j51/prophet_model__9qs0k4/prophet_model-20240824110913.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:09:13 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:09:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
prediction = model1.predict(future_dates)

In [ ]:
trace_open = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat"],
    mode = 'lines',
    name="Forecast"
)
trace_high = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_upper"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Higher uncertainty interval"
)
trace_low = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_lower"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Lower uncertainty interval"
)
trace_close = go.Scatter(
    x = df["ds"],
    y = df["y"],
    name="Data values"
)

#make list for all three scattle objects.
data = [trace_open,trace_high,trace_low,trace_close]
# Construct a new Layout object.
#title - It will display string as a title of graph
layout = go.Layout(title="Power consumption forecasting")
#A list or tuple of trace instances (e.g. [Scatter(…), Bar(…)]) or A single trace instance (e.g. Scatter(…), Bar(…), etc.)
#A list or tuple of dicts of string/value properties where: - The ‘type’ property specifies the trace type.

fig = go.Figure(data=data)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:

fig = go.Figure([go.Scatter(x=df['ds'], y=df['y'],mode='lines',
                    name='Actual')])
#You can add traces using an Express plot by using add_trace
fig.add_trace(go.Scatter(x=prediction['ds'], y=prediction['yhat'],
                   mode='lines+markers',
                    name='predicted'))
#To display a figure using the renderers framework, you call the .show() method on a graph object figure, or pass the figure to the plotly.io.show function.
#With either approach, plotly.py will display the figure using the current default renderer(s).
fig.show()
